In [6]:
from SPARQLWrapper import SPARQLWrapper, JSON
from datetime import datetime
import pandas as pd
import numpy as np

connect to db

In [7]:
sparql = SPARQLWrapper('https://dati.camera.it/sparql')

Create dataframe of the deputati from a legislation

In [8]:
leg_num = 16

In [9]:
dep = pd.read_csv(f'data/dep_{leg_num}.csv')[['persona', 'nomeGruppo', 'nome', 'cognome']]
a = dep['cognome']+ ' ' + dep['nome']
dep['Parlamentare'] = a
dep.drop(['nome', 'cognome'], axis = 1, inplace= True)

a = []
for i in range(dep.shape[0]):
    a.append(dep['persona'][i][39:])

dep['id'] = a
dep.drop('persona', inplace= True, axis = 1)

p = []
dat = []
for i in dep['nomeGruppo']:
    e = i.split()
    p.append(e[-2][1:-1])
    d = e[-1].replace(')', '')
    dat.append(datetime.strptime(d[-10:], '%d.%m.%Y'))

dep['Partito'] = p
dep['dataIngresso'] = dat
dep.drop('nomeGruppo', axis = 1, inplace= True)

first_df = pd.DataFrame(columns= dep.columns)

for i in np.unique(dep['id']):
    if len(dep[dep['id'] == i]) < 2:
        first_df = pd.concat([first_df, dep[dep['id'] == i]])
    else:
        duplicate = dep[dep['id'] == i]
        first_df = pd.concat([first_df, duplicate.head(1)])


In [10]:
sparql.setReturnFormat(JSON)

leg = 'xvi'
year = '2012'
final_df = first_df.copy()

# deputati of a specific legislation
for m in range(1,13):
   print(f'{m:02d}')

   for d in range(1,32):
      month = f'{m:02d}'
      day = f'{d:02d}'
      sparql.setQuery("""
      select distinct ?votazione ?data ?titolo ?numeroVotazione ?cognome ?nome ?espressione ?infoAssenza ?deputato where {
      ?votazione a ocd:votazione;
         dc:date '"""+year+month+day+"""';
         dc:date ?data;
         rdfs:label ?titolo;
         dc:description ?descrizione;
         dc:identifier ?numeroVotazione.


      ## voti espressi
      ?voto a ocd:voto;
         ocd:rif_votazione ?votazione;
         dc:type ?espressione;
         ocd:rif_deputato ?deputato.
      OPTIONAL{?voto dc:description ?infoAssenza}


      ## info deputato
      ?deputato foaf:surname ?cognome; foaf:firstName ?nome  


      } ORDER BY ?cognome
         
      """
      )
      try:
         ret = sparql.queryAndConvert()
         res = ret["results"]["bindings"]
      except Exception as e:
         print(e)

      dep = []
      vot = []
      numero = []
      for i in res:
         dep.append((i.get('deputato')).get('value')[40:46])
         vot.append((i.get('espressione')).get('value'))
         numero.append((i.get('numeroVotazione')).get('value'))

      votazione = pd.DataFrame([dep,vot, numero], index = ['id', 'Voto1', 'numero']).T
         
      for i in list(set(votazione['numero'])):
         espressione = votazione[votazione['numero'] == i][['id', 'Voto1']]
         espressione.columns= ('id', i)
         final_df = pd.merge(final_df, espressione, how = 'left')
            
print(final_df.shape)     
final_df.to_csv('./'+leg+'-years'+'/'+leg+'-'+year+'.csv', index=False)

01
02
03
04
05
06
07
08
09
10
11
12
(681, 4)


In [11]:
print(final_df.shape)     


(681, 4)


Merge

In [12]:
'''
leg = 'xvii'

year = 2013
merged = pd.read_csv(f'./{leg}-years/{leg}-{year}.csv')
print(merged.shape)

for year in [2014,2015,2016,2017,2018]:
    y = pd.read_csv(f'./{leg}-years/{leg}-{year}.csv')
    merged = pd.merge(merged, y, how = 'left')
    print(merged.shape)

merged.to_csv('./'+leg+'-years'+'/'+leg+'-merged.csv', index=False)
'''

"\nleg = 'xvii'\n\nyear = 2013\nmerged = pd.read_csv(f'./{leg}-years/{leg}-{year}.csv')\nprint(merged.shape)\n\nfor year in [2014,2015,2016,2017,2018]:\n    y = pd.read_csv(f'./{leg}-years/{leg}-{year}.csv')\n    merged = pd.merge(merged, y, how = 'left')\n    print(merged.shape)\n\nmerged.to_csv('./'+leg+'-years'+'/'+leg+'-merged.csv', index=False)\n"